In [1]:
import nltk 
import string 
from nltk.corpus import stopwords 

In [2]:

with open('positive_train_new.txt','r',encoding = 'utf8') as inFile, open('Filtered_positive_train_new.txt','w',encoding = 'utf8') as outFile: 
    for line in inFile.readlines(): 
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split()  
            if len(word) >=4 and word not in stopwords.words('english')]), file=outFile) 

In [3]:
with open('not_violent_train.txt','r',encoding = 'utf8') as inFile, open('Filtered_not_violent.txt','w',encoding = 'utf8') as outFile: 
    for line in inFile.readlines(): 
        print(" ".join([word for word in line.lower().translate(str.maketrans('', '', string.punctuation)).split()  
            if len(word) >=4 and word not in stopwords.words('english')]), file=outFile) 

In [4]:
#Importing all the other necessary libraries first
import pandas, xgboost, numpy, textblob, string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

In [5]:
#Reading the dataset
#There are two separate text files, containing the 2 classes of documents
data = open('Filtered_positive_train_new.txt',encoding="utf8").read()
data1 = open('Filtered_not_violent.txt',encoding="utf8").read() 

In [6]:
#There is no separate column denoting the label of each document.
#So adding label to each (Label_1 denotes violent; label_2 denotes violent)
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append("label_1")
#Joining elements from both the classes together into one single file 
    texts.append(" ".join(content[1:]))
for i, line in enumerate(data1.split("\n")):
    content = line.split()
    labels.append("label_2")
    texts.append(" ".join(content[1:]))

In [7]:
#Creating a dataframe using above texts and labels
import pandas
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [9]:
#Now, all instances of one class are clumped together
#Shuffling the rows of the dataframe to avoid discrepancies in future
from sklearn.utils import shuffle
trainDF = shuffle(trainDF)
trainDF

,text,label
414,sacrilege therefore desecration unknown also s...,label_1
2038,order receipt sent prime minister narendra mod...,label_2
605,demandedthe times india j86jcurrent 1956 proqu...,label_1
4240,four monkeys found dead different parts udupi ...,label_2
1804,celebration christian family attacked hindu ra...,label_1
...,...,...
4226,father suffers heart attack incident took plac...,label_2
237,driver mumbai allegedly physically assaulted g...,label_1
2109,veteran industrialist rahul bajaj city attend ...,label_2
3599,victim’s father arrested charge covering tortu...,label_2


In [10]:
data = open('Times_January.txt',encoding="utf8").read()

In [11]:
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append("XX") 
    texts.append(" ".join(content[1:]))

In [12]:
#Creating a dataframe using above texts and labels
import pandas
testDF = pandas.DataFrame()
testDF['text'] = texts
testDF['label'] = labels

In [13]:
testDF

,text,label
0,Tripathy to debut in Sandalwood () Pitobash Tr...,XX
1,isn’t respect or fanfare: Yash on the fan who ...,XX
2,"on ‘noisy’ bikes beat up 2 policemen | Jan 12,...",XX
3,Ali Khan soaking in the sun will make you beam...,XX
4,rescues stranded tourists from north Sikkim in...,XX
...,...,...
13854,that propelled BJP to power in 2014 has dissip...,XX
13855,Delhi: Accused Mehtab arrested day after he ki...,XX
13856,"Nath may have been framed in rape case, observ...",XX
13857,"‘Can’t remove CM over pending cases’ | Jan 9, ...",XX


In [14]:
testDF1= testDF.iloc[0:800]
result = trainDF.append(testDF1)
result

,text,label
414,sacrilege therefore desecration unknown also s...,label_1
2038,order receipt sent prime minister narendra mod...,label_2
605,demandedthe times india j86jcurrent 1956 proqu...,label_1
4240,four monkeys found dead different parts udupi ...,label_2
1804,celebration christian family attacked hindu ra...,label_1
...,...,...
795,Yadav to launch poll on twitter LUCKNOW: Twitt...,XX
796,Gandhi to hold Kisan Abhar rally in Madhya Pra...,XX
797,"man dies of injuries in car accident | Jan 11,...",XX
798,"man sets self afire after tiff with wife, dies...",XX


In [15]:
trainDF = result
#Splitting the dataset into training(80%) and validation(20%) sets  
import numpy as np
from sklearn.model_selection import train_test_split
train_x, train_y=trainDF['text'].iloc[0:4992], trainDF['label'].iloc[0:4992]
valid_x, valid_y=trainDF['text'].iloc[4992:], trainDF['label'].iloc[4992:]

In [17]:
valid_y

0      XX
1      XX
2      XX
3      XX
4      XX
       ..
795    XX
796    XX
797    XX
798    XX
799    XX
Name: label, Length: 800, dtype: object

In [18]:
train_y

414     label_1
2038    label_2
605     label_1
4240    label_2
1804    label_1
         ...   
4226    label_2
237     label_1
2109    label_2
3599    label_2
3845    label_2
Name: label, Length: 4992, dtype: object

In [19]:
#label encoding the class-labels or the target variable 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [20]:
#The next step is the feature engineering step. 
#Raw text data will be transformed into feature vectors and new features will be created using the existing dataset.

#count vectorizer, tf-idf, word embeddings, topic models, part-of-speech tagging,

#Creating count vectorizer object and using it to transform data
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [21]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=3000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

C:\Users\ANKAN\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [22]:
#Using a pre-trained word embedding to create a mapping of token and their respective embeddings
import numpy
embeddings_index = {}
for i, line in enumerate(open('C:/Users/ANKAN/Downloads/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec',encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [24]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


#Creatng a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

#Converting text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=50)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=50)

Using TensorFlow backend.


In [25]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
#Feature engineering - introducing new features that are expected to be useful
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [27]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

In [28]:
# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [29]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ANKAN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [31]:
trainDF.tail(10)

,text,label,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,noun_count,verb_count,adj_count,adv_count,pron_count
790,noise & gridlock capital of UP LUCKNOW: The ef...,XX,2221,365,6.068306,55,45,14,158,57,21,8,5
791,"mothers to get Rs 10,000 for girl child | Jan ...",XX,982,165,5.915663,19,12,2,60,19,15,1,4
792,one of the oldest leopards dies at Van Vihar |...,XX,1539,285,5.381119,35,48,6,89,45,32,12,16
793,members wrangle over quantum of punishment for...,XX,4557,794,5.732075,105,133,21,293,170,45,32,32
794,"son starts charity | Jan 11, 2019, 08:35 IST S...",XX,636,102,6.174757,15,20,3,43,13,6,4,8
795,Yadav to launch poll on twitter LUCKNOW: Twitt...,XX,1095,173,6.293103,20,34,4,66,29,15,5,10
796,Gandhi to hold Kisan Abhar rally in Madhya Pra...,XX,2164,369,5.848649,37,72,6,160,65,13,18,3
797,"man dies of injuries in car accident | Jan 11,...",XX,1126,189,5.926316,27,29,5,67,38,9,4,8
798,"man sets self afire after tiff with wife, dies...",XX,787,143,5.465278,23,22,2,47,27,5,4,15
799,"Plateau banks on tankers, bore wells PANAJI: F...",XX,3289,586,5.603066,59,66,10,169,112,35,31,31


In [32]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [33]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [34]:
from sklearn.model_selection import cross_val_score

# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("Naive Bayes Count Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Naive Bayes WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Naive Bayes N-Gram Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Naive Bayes CharLevel Vectors: ", accuracy)
scores = cross_val_score(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')


Naive Bayes Count Vectors:  0.995
10 fold Cross-validated score:  0.9775659318637275 

Naive Bayes WordLevel TF-IDF:  0.975
10 fold Cross-validated score:  0.9829743486973946 

Naive Bayes N-Gram Vectors:  1.0
10 fold Cross-validated score:  0.8946336673346693 

Naive Bayes CharLevel Vectors:  1.0
10 fold Cross-validated score:  0.9527234468937875 



In [35]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(max_iter=4000), xtrain_count, train_y, xvalid_count)
print ("Logistic Regression, Count Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Logistic Regression, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Logistic Regression, N-Gram Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Logistic Regression, CharLevel Vectors: ", accuracy)
scores = cross_val_score(linear_model.LogisticRegression(max_iter=4000), xtrain_tfidf_ngram_chars, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Logistic Regression, Count Vectors:  0.02875
10 fold Cross-validated score:  0.9855779559118236 

Logistic Regression, WordLevel TF-IDF:  0.0725
10 fold Cross-validated score:  0.9899851703406813 

Logistic Regression, N-Gram Vectors:  0.94375
10 fold Cross-validated score:  0.9535266533066131 

Logistic Regression, CharLevel Vectors:  0.51375
10 fold Cross-validated score:  0.982972745490982 



In [36]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)
scores = cross_val_score(svm.SVC(), xtrain_tfidf_ngram, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

SVM, N-Gram Vectors:  0.86875
10 fold Cross-validated score:  0.9669478957915831 



In [37]:
# Random forests on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("Random Forests, Count Vectors: ", accuracy)
scores = cross_val_score(ensemble.RandomForestClassifier(), xtrain_count, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Random Forests, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Random Forests, Count Vectors:  0.18
10 fold Cross-validated score:  0.9733583166332664 

Random Forests, WordLevel TF-IDF:  0.07375
10 fold Cross-validated score:  0.9839747494989981 



In [38]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)
scores = cross_val_score(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, cv=10)
print ("10 fold Cross-validated score: ", np.mean(scores), '\n')

Xgb, Count Vectors:  0.08625
10 fold Cross-validated score:  0.987182765531062 

Xgb, WordLevel TF-IDF:  0.03625
10 fold Cross-validated score:  0.9851783567134268 

Xgb, CharLevel Vectors:  0.1075
10 fold Cross-validated score:  0.9837743486973949 



In [39]:
from sklearn.ensemble import RandomForestClassifier
rmf = RandomForestClassifier()
rmf.fit(xtrain_tfidf, train_y)

RandomForestClassifier()

In [40]:
y_pred=rmf.predict(xvalid_tfidf)

In [41]:
y_pred

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [43]:
testDF1['predicted']=y_pred

<ipython-input-43-f2ab29ad97b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testDF1['predicted']=y_pred


In [45]:
testDF1

,text,label,predicted
0,Tripathy to debut in Sandalwood () Pitobash Tr...,XX,1
1,isn’t respect or fanfare: Yash on the fan who ...,XX,1
2,"on ‘noisy’ bikes beat up 2 policemen | Jan 12,...",XX,0
3,Ali Khan soaking in the sun will make you beam...,XX,1
4,rescues stranded tourists from north Sikkim in...,XX,1
...,...,...,...
795,Yadav to launch poll on twitter LUCKNOW: Twitt...,XX,1
796,Gandhi to hold Kisan Abhar rally in Madhya Pra...,XX,1
797,"man dies of injuries in car accident | Jan 11,...",XX,1
798,"man sets self afire after tiff with wife, dies...",XX,1


In [46]:
testDF1.to_csv('predicted_stage_1.csv')